In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/siddarthg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/siddarthg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize

Using TensorFlow backend.


In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_data.shape

(7613, 5)

In [6]:
train_data.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
train_data.location.value_counts()

USA                         104
New York                     71
United States                50
London                       45
Canada                       29
                           ... 
WAISTDEEP, TX                 1
Miami ??                      1
Studio                        1
 BC, US, Asia or Europe.      1
y/e/l                         1
Name: location, Length: 3341, dtype: int64

In [8]:
train_data.keyword.value_counts()

fatalities               45
armageddon               42
deluge                   42
body%20bags              41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [9]:
test_data.shape

(3263, 4)

In [11]:
test_data.location.value_counts()

New York                               38
USA                                    37
Worldwide                              16
United States                          15
Canada                                 13
                                       ..
South East Florida                      1
http://www.amazon.com/dp/B00HR          1
Albequerque                             1
Greater Vancouver, British Columbia     1
y/e/l                                   1
Name: location, Length: 1602, dtype: int64

In [10]:
%%time
train_data['textnew'] = train_data['text'].apply(lambda s : clean(s))
test_data['textnew'] = test_data['text'].apply(lambda s : clean(s))

CPU times: user 4min 37s, sys: 0 ns, total: 4min 37s
Wall time: 4min 37s


In [12]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [13]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_data.target.values)
print(y)

[1 1 1 ... 1 1 1]


In [14]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_data['text'], y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
xpred = test_data['text']

In [15]:
print("Train Shape: ",xtrain.shape,
      "\nTrain y shape: ",ytrain.shape,
      "\nValidation Train: ",xvalid.shape,
      "\nValidation y shape: ",yvalid.shape)

Train Shape:  (6851,) 
Train y shape:  (6851,) 
Validation Train:  (762,) 
Validation y shape:  (762,)


In [16]:
#TFIDF model
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [17]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.472 


In [18]:

ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [19]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.454 


In [20]:
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.455 


In [21]:
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.222 


In [22]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [23]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.492 


In [24]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.476 


In [25]:


# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))



logloss: 0.465 


In [26]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.537 


In [27]:


# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))



logloss: 0.570 


In [28]:
embeddings_index = {}
f = open(r'glove.840B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


print('Found %s word vectors.' % len(embeddings_index))

Found 2195892 word vectors.


In [29]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [30]:

stop_words = stopwords.words('english')
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [ sent2vec(x) for x in tqdm(xvalid)]
xpred_glove = [sent2vec(x) for x in tqdm(xpred)]

100%|██████████| 3263/3263 [00:00<00:00, 4151.15it/s]


In [31]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)
xpred_glove = np.array(xpred_glove)

In [32]:
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.510 


In [33]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=250, colsample_bytree=0.8, 
                        subsample=0.75, nthread=10, learning_rate=0.05, silent=True)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.428 


In [34]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.75, nthread=10, learning_rate=0.1, silent=True)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.444 


In [31]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [32]:
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [33]:
model = Sequential()

model.add(Dense(500, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(500, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('sigmoid'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [34]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=10, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 6851 samples, validate on 762 samples
Epoch 1/10
6851/6851 [==============================] - 1s 184us/step - loss: 0.5154 - val_loss: 0.4197
Epoch 2/10
6851/6851 [==============================] - 0s 70us/step - loss: 0.4091 - val_loss: 0.4152
Epoch 3/10
6851/6851 [==============================] - 1s 75us/step - loss: 0.3599 - val_loss: 0.4222
Epoch 4/10
6851/6851 [==============================] - 0s 72us/step - loss: 0.3279 - val_loss: 0.4489
Epoch 5/10
6851/6851 [==============================] - 0s 70us/step - loss: 0.3014 - val_loss: 0.4803
Epoch 6/10
6851/6851 [==============================] - 0s 73us/step - loss: 0.2824 - val_loss: 0.5050
Epoch 7/10
6851/6851 [==============================] - 0s 70us/step - loss: 0.2635 - val_loss: 0.5081
Epoch 8/10
6851/6851 [==============================] - 0s 72us/step - loss: 0.2375 - val_loss: 0.5343
Epoch 9/10
6851/6851 [==============================] - 0s 71us/step - loss: 0.2390 - val_loss: 0.5253
Epoch 10/10
6851/6851 [==

In [35]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xpred_seq = token.texts_to_sequences(xpred)
# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xpred_pad = sequence.pad_sequences(xpred_seq, maxlen=max_len)
word_index = token.word_index

In [36]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 22700/22700 [00:00<00:00, 500064.08it/s]


In [37]:
# # A simple LSTM with glove embeddings and two dense layers
# model = Sequential()
# model.add(Embedding(len(word_index) + 1,
#                      300,
#                      weights=[embedding_matrix],
#                      input_length=max_len,
#                      trainable=False))
# model.add(SpatialDropout1D(0.3))
# model.add(LSTM(200, dropout=0.3, recurrent_dropout=0.3))

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.45))

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.45))

# model.add(Dense(2))
# model.add(Activation('sigmoid'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:
# model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=15, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

In [53]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.2, recurrent_dropout=0.2)))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.8))
 
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3,mode='auto', verbose=1)
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=30, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])
preds=model.predict(xpred_pad)

Train on 6851 samples, validate on 762 samples
Epoch 1/30
6851/6851 [==============================] - 6s 925us/step - loss: 0.6580 - accuracy: 0.6112 - val_loss: 0.5856 - val_accuracy: 0.6549
Epoch 2/30
6851/6851 [==============================] - 5s 694us/step - loss: 0.5481 - accuracy: 0.7403 - val_loss: 0.4795 - val_accuracy: 0.8202
Epoch 3/30
6851/6851 [==============================] - 5s 693us/step - loss: 0.4855 - accuracy: 0.7846 - val_loss: 0.4362 - val_accuracy: 0.8084
Epoch 4/30
6851/6851 [==============================] - 5s 696us/step - loss: 0.4712 - accuracy: 0.7946 - val_loss: 0.4222 - val_accuracy: 0.8202
Epoch 5/30
6851/6851 [==============================] - 5s 700us/step - loss: 0.4685 - accuracy: 0.7932 - val_loss: 0.4183 - val_accuracy: 0.8241
Epoch 6/30
6851/6851 [==============================] - 5s 707us/step - loss: 0.4562 - accuracy: 0.7987 - val_loss: 0.4253 - val_accuracy: 0.8215
Epoch 7/30
6851/6851 [==============================] - 5s 711us/step - loss:

In [54]:
preds

array([[0.3717112 , 0.6331731 ],
       [0.19446611, 0.86032313],
       [0.42670214, 0.63622445],
       ...,
       [0.18332824, 0.871254  ],
       [0.46379244, 0.56549716],
       [0.47447062, 0.54128635]], dtype=float32)

In [55]:
p=[]
for i in preds:
    if i[0]>i[1]:
        p.append(0)
    else:
        p.append(1)

In [56]:
p

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [57]:
p=np.array(p)

In [58]:
cnt0=0;cnt1=0
for i in p:
    if i==0:
        cnt0+=1
    else:
        cnt1+=1
print(cnt0,"\n",cnt1)

2143 
 1120


In [59]:
test_df=pd.read_csv('sample_submission.csv')

In [60]:
test_df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [61]:
test_df.drop(columns = 'target' , inplace=True)
test_df.head()

,id
0,0
1,2
2,3
3,9
4,11


In [62]:
test_df['target']= p
test_df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [63]:
test_df.target.value_counts

<bound method IndexOpsMixin.value_counts of 0       1
1       1
2       1
3       1
4       1
       ..
3258    1
3259    1
3260    1
3261    1
3262    1
Name: target, Length: 3263, dtype: int64>

In [64]:
test_df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [65]:
compression_opts = dict(method='zip',

                        archive_name='out5.csv')  

test_df.to_csv('out.zip', index=False,

          compression=compression_opts)  

In [66]:
!ls

glove.840B.300d.txt  out2.csv  out.csv		      test.csv
glove.840B.300d.zip  out3.csv  out.zip		      train.csv
out1.csv	     out4.csv  sample_submission.csv  Untitled.ipynb
